# Transform CoLoRE outputs to a file that can be used by desisim

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import fitsio
import matplotlib.pyplot as plt

In [3]:
#Open data file (test file with few quasars)
h = fitsio.FITS('../example_data/raw_colore/N1000_out_srcs_s0_15.fits')

In [4]:
# get information about quasars (TYPE,RA,DEC,Z_COSMO,DZ_RSD)
catalog = h[1].read()
z_qso = catalog['Z_COSMO']
Nq = len(z_qso)
print('# quasars =',Nq)
print(np.min(z_qso),'< z_qso <',np.max(z_qso))

# quasars = 1000
0.139725 < z_qso < 3.48193


In [5]:
# get arraw with redshift in each cell of grid
z = h[4].read()['Z']
Nz=len(z)

In [6]:
# Get deltas (fluctuation around mean density) and line of sight velocity (actually, dz)
delta = h[2].read()
velocity = h[3].read()
# Convert density to flux
import lya_mock_functions as mock
tau = mock.get_tau(z,1+delta)
flux = np.exp(-tau)

In [7]:
filename='desisim_lya_N'+str(Nq)+'.fits'

In [8]:
fits = fitsio.FITS(filename,'rw',clobber=True)

In [9]:
# write skewers for desisim
for i in range(Nq):
    # only add absorption in the forest 
    no_forest = (z > z_qso[i])
    flux[i][no_forest]=1.0
    data = {}
    data['LAMBDA']=1215.67*(1+z)
    data['FLUX']=flux[i]
    head = {}
    head['ZQSO']=z_qso[i]
    head['RA']=catalog['RA'][i]
    head['DEC']=catalog['DEC'][i]
    head['MAG_G']=22
    fits.write(data,header=head)
fits.close()

In [10]:
# test whether we can read file as in desisim.lya_spectra
test = fitsio.FITS(filename)
heads = []
for i in range(Nq):
    heads.append(test[i+1].read_header())    

zqso = np.array([head['ZQSO'] for head in heads])
ra = np.array([head['RA'] for head in heads])
dec = np.array([head['DEC'] for head in heads])
mag_g = np.array([head['MAG_G'] for head in heads])

for i in range(Nq):
    data = test[i + 1].read()
    la = data['LAMBDA'][:]
    tr = data['FLUX'][:]

# close FITS file
fits.close()